## Importing Basic libraries

In [1]:
import os
import zipfile
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Importing the Data

In [3]:
zip=zipfile.ZipFile("/content/101_food_classes_10_percent.zip")
zip.extractall()
zip.close()

## Exploring the data

In [4]:
for path,folder,filename in os.walk("/content/101_food_classes_10_percent"):
  print(f"{len(filename)} in {path}")

0 in /content/101_food_classes_10_percent
0 in /content/101_food_classes_10_percent/train
75 in /content/101_food_classes_10_percent/train/spring_rolls
75 in /content/101_food_classes_10_percent/train/dumplings
75 in /content/101_food_classes_10_percent/train/falafel
75 in /content/101_food_classes_10_percent/train/hot_and_sour_soup
75 in /content/101_food_classes_10_percent/train/crab_cakes
75 in /content/101_food_classes_10_percent/train/red_velvet_cake
75 in /content/101_food_classes_10_percent/train/oysters
75 in /content/101_food_classes_10_percent/train/cannoli
75 in /content/101_food_classes_10_percent/train/strawberry_shortcake
75 in /content/101_food_classes_10_percent/train/apple_pie
75 in /content/101_food_classes_10_percent/train/samosa
75 in /content/101_food_classes_10_percent/train/greek_salad
75 in /content/101_food_classes_10_percent/train/filet_mignon
75 in /content/101_food_classes_10_percent/train/tiramisu
75 in /content/101_food_classes_10_percent/train/hamburger
7

## Image preprocessing

In [5]:
# importing required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# creating imagedatagenerator instance with scaling
train_data_scale=ImageDataGenerator(rescale=1/255)
test_data_scale=ImageDataGenerator(rescale=1/255)

In [7]:
#preprocessing image for model

train_data=train_data_scale.flow_from_directory("/content/101_food_classes_10_percent/train",
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode="categorical")

test_data=train_data_scale.flow_from_directory("/content/101_food_classes_10_percent/test",
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode="categorical")

Found 7575 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


## Model Building

In [8]:
# Transfer learning
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
base_model.trainable = False

16705208/16705208 [==============================] - 2s 0us/step


In [9]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
base = base_model(inputs, training=False)
pooled = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling")(base)
outputs = tf.keras.layers.Dense(101, activation="softmax", name="output_layer")(pooled)
model = tf.keras.Model(inputs, outputs)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling (Glo  (None, 1280)             0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainable params: 4,049,571
_________________________________________________________________


In [11]:
# Compile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [12]:
# Fit
history = model.fit(train_data,
                    epochs=5,
                    validation_data=test_data)


Epoch 1/5
237/237 [==============================] - 114s 412ms/step - loss: 4.7187 - accuracy: 0.0099 - val_loss: 4.7030 - val_accuracy: 0.0099
Epoch 2/5
237/237 [==============================] - 112s 472ms/step - loss: 4.7069 - accuracy: 0.0090 - val_loss: 4.6997 - val_accuracy: 0.0099
Epoch 3/5
237/237 [==============================] - 99s 421ms/step - loss: 4.7141 - accuracy: 0.0088 - val_loss: 4.6933 - val_accuracy: 0.0101
Epoch 4/5
237/237 [==============================] - 105s 445ms/step - loss: 4.7052 - accuracy: 0.0098 - val_loss: 4.7018 - val_accuracy: 0.0100
Epoch 5/5
237/237 [==============================] - 94s 396ms/step - loss: 4.7139 - accuracy: 0.0083 - val_loss: 4.6837 - val_accuracy: 0.0099


In [13]:
# Evaluation
model.evaluate(test_data)

790/790 [==============================] - 77s 97ms/step - loss: 4.6837 - accuracy: 0.0099


[4.683703899383545, 0.00994059443473816]

## Fine Tuning

In [14]:
# freezing all layers except last five layers
base_model.trainable = True
for layer in base_model.layers[:-5]:
  layer.trainable = False

In [15]:
for layer_number, layer in enumerate(base_model.layers):
  print(layer_number, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 rescaling_1 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze False
39

In [16]:
history_1 = model.fit(train_data,
                    epochs=5,
                    validation_data=test_data)

Epoch 1/5
237/237 [==============================] - 95s 401ms/step - loss: 4.7034 - accuracy: 0.0091 - val_loss: 4.6858 - val_accuracy: 0.0099
Epoch 2/5
237/237 [==============================] - 95s 400ms/step - loss: 4.6991 - accuracy: 0.0084 - val_loss: 4.6867 - val_accuracy: 0.0099
Epoch 3/5
237/237 [==============================] - 96s 407ms/step - loss: 4.7051 - accuracy: 0.0090 - val_loss: 4.6956 - val_accuracy: 0.0116
Epoch 4/5
237/237 [==============================] - 95s 400ms/step - loss: 4.7036 - accuracy: 0.0098 - val_loss: 4.6836 - val_accuracy: 0.0099
Epoch 5/5
237/237 [==============================] - 97s 409ms/step - loss: 4.7014 - accuracy: 0.0121 - val_loss: 4.6888 - val_accuracy: 0.0099


In [17]:
model_aug=tf.keras.models.Sequential([tf.keras.layers.RandomFlip(),
                                     tf.keras.layers.RandomRotation((0.2,0.9))])

In [19]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
aug=model_aug(inputs)
base = base_model(aug, training=False)
pooled = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling")(base)
outputs = tf.keras.layers.Dense(101, activation="softmax", name="output_layer")(pooled)
model_1 = tf.keras.Model(inputs, outputs)

In [22]:
# Compile
model_1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
history_2 = model_1.fit(train_data,
                    epochs=5,
                    validation_data=test_data)

Epoch 1/5
237/237 [==============================] - 110s 425ms/step - loss: 4.6360 - accuracy: 0.0086 - val_loss: 4.6154 - val_accuracy: 0.0099
Epoch 2/5
237/237 [==============================] - ETA: 0s - loss: 4.6167 - accuracy: 0.0084